In [1]:
from ase import Atoms, io
import torch
import schnetpack as spk
from schnetpack.data import AtomsDataModule
from schnetpack.datasets import MD17
import schnetpack.transform as trn

device = torch.device("cuda")

In [2]:
model_path = "best_model"
best_model = torch.load(model_path, map_location=device)

In [13]:
"""
data = AtomsDataModule(
    './tob9.db',
    batch_size=10,
    num_train=950,
    num_val=50,
    transforms=[
        trn.ASENeighborList(cutoff=4.),
        trn.RemoveOffsets(MD17.energy, remove_mean=True, remove_atomrefs=False),
        trn.CastTo32()
    ],
    num_workers=1,
    pin_memory=True, # set to false, when not using a GPU
)
data.prepare_data()
data.setup()
"""

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 95/95 [00:01<00:00, 64.08it/s]


In [16]:
"""
structure = data.test_dataset[0]
atoms = Atoms(
    numbers=structure[spk.properties.Z], positions=structure[spk.properties.R]
)
io.write('tob9.xyz', atoms, format='xyz')
"""

In [3]:
molecule_path = 'tob9.xyz'
ase_dir = './'
model_path = "best_model"

tob9_ase = spk.interfaces.AseInterface(
    molecule_path,
    ase_dir,
    model_file=model_path,
    neighbor_list=trn.ASENeighborList(cutoff=4.0),
    energy_key=MD17.energy,
    force_key=MD17.forces,
    energy_unit="eV",
    position_unit="Ang",
    device="cuda",
    dtype=torch.float64,
)

INFO:schnetpack.interfaces.ase_interface:Loading model from best_model


In [4]:
tob9_ase.optimize(fmax=1e-2)

                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 01:14:46     -452.860487        4.1414
BFGSLineSearch:    1[  2] 01:14:47     -454.510691        1.8411
BFGSLineSearch:    2[  4] 01:14:48     -455.433983        1.4856
BFGSLineSearch:    3[  6] 01:14:49     -455.736843        1.3235
BFGSLineSearch:    4[  8] 01:14:50     -455.955815        0.8622
BFGSLineSearch:    5[ 10] 01:14:51     -456.067281        0.5758
BFGSLineSearch:    6[ 12] 01:14:52     -456.175353        0.7863
BFGSLineSearch:    7[ 14] 01:14:53     -456.260972        0.4243
BFGSLineSearch:    8[ 16] 01:14:53     -456.303085        0.3877
BFGSLineSearch:    9[ 18] 01:14:54     -456.346889        0.4145
BFGSLineSearch:   10[ 20] 01:14:55     -456.374827        0.3673
BFGSLineSearch:   11[ 22] 01:14:56     -456.400670        0.3671
BFGSLineSearch:   12[ 24] 01:14:57     -456.419466        0.2414
BFGSLineSearch:   13[ 26] 01:14:58     -456.434710        0.1982
BFGSLineSearch:   14[ 28]

In [ ]:
molecule_path = 'optimization.extxyz'
ase_dir = './'
model_path = "best_model"

tob9_op_ase = spk.interfaces.AseInterface(
    molecule_path,
    ase_dir,
    model_file=model_path,
    neighbor_list=trn.ASENeighborList(cutoff=4.0),
    energy_key=MD17.energy,
    force_key=MD17.forces,
    energy_unit="eV",
    position_unit="Ang",
    device="cuda",
    dtype=torch.float64,
)

!rm './simulation_300K.log'
!rm './simulation_300K.traj'

tob9_op_ase.init_md(
    'simulation_300K',
    time_step=0.5, #fs
    temp_init=300,
    temp_bath=300, #NVT None:NVE
    reset=True,
    interval=100
)

tob9_op_ase.run_md(1000000)

INFO:schnetpack.interfaces.ase_interface:Loading model from best_model
/home/zkm/.local/lib/python3.8/site-packages/ase/md/md.py:48: FutureWarning: Specify the temperature in K using the 'temperature_K' argument
  warnings.warn(FutureWarning(w))
